# Policy generation

In [1]:
# Imports

from __future__ import print_function

import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [4]:
# Hyperparameters
batch_size = 1000   # num of training examples to train on in a batch
limit_timesteps = 15 # sequence length
num_hidden_layers = 51
state_dim = 3 # dimensions of the state variable passed as input to the LSTM model
future = 5

In [5]:
# Sequence model

class Sequence(nn.Module):
    def __init__(self, state_dim, num_hidden_layers):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(state_dim, num_hidden_layers)
        self.lstm2 = nn.LSTMCell(num_hidden_layers, num_hidden_layers)
        self.linear = nn.Linear(num_hidden_layers, state_dim)

    def forward(self, input, future = 0):
        outputs = []
        
        h_t = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        c_t = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(input.size(0), num_hidden_layers).double(), requires_grad=False)
        
        # run 19 times
        # input_t.size = 5 x 5
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            input_t = input_t.squeeze(1)
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            if (future == 0):
                outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        print(outputs.size())
        return outputs

In [6]:
# Load model only
seq = Sequence(state_dim, num_hidden_layers)
params = torch.load('ACAS_theparameters.pt')
seq.load_state_dict(params)
seq.double()

Sequence (
  (lstm1): LSTMCell(3, 51)
  (lstm2): LSTMCell(51, 51)
  (linear): Linear (51 -> 3)
)

In [7]:
# load data and make training set
data = pd.read_csv('cleaned/states_2017-08-28-00.csv')

# # drop unneeded columns to form matrix to be trained on
# data_matr = data.drop(['NMAC_id', 'alert_1', 'alert_2', 'onground_1', 'onground_2'], axis=1).as_matrix().astype('float')

# restructure data
all_planes = []

count = 0
unique_icao = data.icao24.unique()
for i, name in enumerate(unique_icao):
    if count % 100 == 0: print(count)
    # only get batch size for now
    if count >= batch_size:
        break
    
    # get rows where icao == name (boolean vector)
    selector = data['icao24'] == name
    one_plane_df = data[selector]
    
    one_plane_df = one_plane_df.drop(['icao24','time','lat','lon','velocity','heading'], axis=1)
    one_plane_arr = one_plane_df.as_matrix()
    one_plane_arr = one_plane_arr[0:limit_timesteps]
    
    all_planes.append(one_plane_arr)
    
    count += 1

batch = np.asarray(all_planes) # convert to np matrix

0
100
200
300
400
500
600
700
800
900
1000
